In [ ]:
from pydantic import BaseModel, field_validator
from typing import Any
from PIL import Image
import numpy as np
import io

class ImageRGBModel(BaseModel):
    image: Any  # Accepte n'importe quel type, mais on valide ensuite

    @field_validator("image")
    @classmethod
    def validate_image(cls, value):
        """Vérifie que l'entrée est une image RGB."""
        if isinstance(value, bytes):
            try:
                value = Image.open(io.BytesIO(value))  # Ouvrir depuis un flux binaire
            except Exception:
                raise ValueError("Impossible de charger l'image à partir des bytes.")
        
        if not isinstance(value, Image.Image):
            raise ValueError("L'entrée doit être une image PIL.")
        
        if value.mode != "RGB":
            raise ValueError("L'image doit être en mode RGB.")
        
        return value

# Test
try:
    img = Image.open("mon_image.jpg")  # Charger une image
    data = ImageRGBModel(image=img)  # Validation avec Pydantic
    print("Image valide :", data.image.size)
except Exception as e:
    print("Erreur :", e)


In [ ]:
class ImageRGBModel(BaseModel):
    image: Any  # Accepte des images sous différents formats

    @field_validator("image")
    @classmethod
    def validate_image(cls, value):
        """Vérifie que l'entrée est une image RGB sous forme de PIL ou NumPy."""
        if isinstance(value, bytes):
            try:
                value = Image.open(io.BytesIO(value))
            except Exception:
                raise ValueError("Impossible de charger l'image à partir des bytes.")

        elif isinstance(value, np.ndarray):
            if value.ndim != 3 or value.shape[2] != 3:
                raise ValueError("L'image doit être en RGB (H, W, 3).")
            value = Image.fromarray(value)  # Convertir en PIL pour uniformiser
        
        if not isinstance(value, Image.Image):
            raise ValueError("L'entrée doit être une image PIL ou un array NumPy.")
        
        if value.mode != "RGB":
            raise ValueError("L'image doit être en mode RGB.")

        return value


In [ ]:
from fastapi import FastAPI, UploadFile, File
from pydantic import ValidationError

app = FastAPI()

@app.post("/upload-image/")
async def upload_image(file: UploadFile = File(...)):
    try:
        img = Image.open(file.file)  # Charger l’image
        data = ImageRGBModel(image=img)  # Valider avec Pydantic
        return {"message": "Image valide", "size": data.image.size}
    except ValidationError as e:
        return {"error": e.errors()}
    except Exception as e:
        return {"error": str(e)}
